In [2]:
import torch
import torch.nn as nn
import torch.nn.init
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import numpy as np
import pandas as pd
import glob
import os
from PIL import Image

### Preparation of dataset

In [8]:
def create_dataset(texts_path, images_path):
  """Create dataframe with path to image and corresponding text description"""

  texts_df = pd.read_csv(texts_path)
  texts_df['text'] = texts_df['color'] + " " + texts_df['name'] + " " + texts_df['description']
  texts_df = texts_df[['Unnamed: 0','text']]
  
  df = pd.DataFrame(columns=["Image","Text"])  
    
  for image in glob.glob(images_path):
    img_name = os.path.basename(image)
    key_img_name = img_name.split('_')[0]
    img_descr = texts_df[texts_df['Unnamed: 0']==int(key_img_name)].iloc[0,-1]
    df = df.append({'Image': img_name, 'Text':img_descr}, ignore_index=True)
    
  return df

In [9]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations_file
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = Image.open(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Transformation of images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(224,224)),
    transforms.Normalize((0.5,), (0.5,))  # Scale images to [-1, 1]
])

# Transformation of texts
target_transform = transforms.Compose([
    transforms.ToTensor(),  # Transform to tensor
    transforms.Resize(size=(224,224)),
    transforms.Normalize((0.5,), (0.5,))  # Scale images to [-1, 1]
])

# Data paths
descriptions_data = "./processed_data/processedSKUs_nodups.csv"
images_folder = "./processed_data/images/*.jpg"
img_dir = "./processed_data/images/"

# Creation of organized dataframe
annotations_file = create_dataset(descriptions_data, images_folder)

# Creation of custom Dataset
dataset = CustomImageDataset(annotations_file, img_dir, transform=transform)

#trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
#testloader = DataLoader(testset, batch_size=5, shuffle=False)

In [12]:
next(iter(dataset))

(tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]),
 '18 Kt Rose Gold Supreme Swan Charm Bracelet The Supreme Swan Charm Bracelet is the symbol of love, peace and grace. Gift this to a loved one that adds these elements to your life. Made in 